In [1]:
import pandas as pd

In [2]:
# Load all Data
btc = pd.read_csv('../Cleaned_Data/BTC-USD-TI.csv')
gtrends = pd.read_csv('../Cleaned_Data/Adjusted_Gtrends.csv')
gnews = pd.read_csv('../Cleaned_Data/Gnews_sentiment_daily.csv')
macro = pd.read_csv('../Cleaned_Data/merged_macrodata.csv')
block = pd.read_csv('../Cleaned_Data/Blockchain_data.csv')

In [3]:
btc_gtrends = pd.merge(btc, gtrends, on='Date', how='left')
btc_gtrends_gnews = pd.merge(btc_gtrends, gnews, on='Date', how='left')
btc_gtrends_gnews_macro = pd.merge(btc_gtrends_gnews, macro, on='Date', how='left')
btc_gtrends_gnews_macro_block = pd.merge(btc_gtrends_gnews_macro, block, on='Date', how='left')


# Filter the df to have start date of 2014-10-31 and end date of 2024-03-31
btc_gtrends_gnews_macro_block = btc_gtrends_gnews_macro_block[(btc_gtrends_gnews_macro_block['Date'] >= '2014-10-31') & (btc_gtrends_gnews_macro_block['Date'] <= '2024-03-31')]


In [4]:
# Make sure all columns are numeric
for column in btc_gtrends_gnews_macro_block.columns:
    if column != 'Date':
        btc_gtrends_gnews_macro_block[column] = pd.to_numeric(btc_gtrends_gnews_macro_block[column], errors='coerce')

# Fill missing values for weekend days with previous friday's values
btc_gtrends_gnews_macro_block = btc_gtrends_gnews_macro_block.fillna({'Interest_Rate': btc_gtrends_gnews_macro_block['Interest_Rate'].ffill(), 
                                                          'Price_NYSE': btc_gtrends_gnews_macro_block['Price_NYSE'].ffill(), 
                                                          'Price_SP500': btc_gtrends_gnews_macro_block['Price_SP500'].ffill(), 
                                                          'Price_NASDAQ': btc_gtrends_gnews_macro_block['Price_NASDAQ'].ffill(), 
                                                          'Price_gold': btc_gtrends_gnews_macro_block['Price_gold'].ffill(), 
                                                          'Price_oil': btc_gtrends_gnews_macro_block['Price_oil'].ffill()})

#Fill missing values via interpolation for Sentiment and Blockchain metrics data
btc_gtrends_gnews_macro_block['Sentiment_Bullish'] = btc_gtrends_gnews_macro_block['Sentiment_Bullish'].interpolate(method='linear')
btc_gtrends_gnews_macro_block['users'] = btc_gtrends_gnews_macro_block['users'].interpolate(method='linear')
btc_gtrends_gnews_macro_block['hash_rate'] = btc_gtrends_gnews_macro_block['hash_rate'].interpolate(method='linear')

# Fill first entries for User and Hash Rate with the first non-NaN value
btc_gtrends_gnews_macro_block = btc_gtrends_gnews_macro_block.fillna({'users': btc_gtrends_gnews_macro_block['users'].bfill(), 'hash_rate': btc_gtrends_gnews_macro_block['hash_rate'].bfill()})

btc_gtrends_gnews_macro_block.reset_index(drop=True, inplace=True)



btc_gtrends_gnews_macro_block


,Date,Open,High,Low,Close,Volume,RSI,ATR,MACD,MFI,...,GTrends_Interest,Sentiment_Bullish,Price_oil,Price_gold,Price_NASDAQ,Price_SP500,Price_NYSE,Interest_Rate,hash_rate,users
0,2014-10-31,345.009003,348.045013,337.141998,338.321014,12545400,36.538491,17.134664,-12.367111,44.185954,...,1.77,54.195092,80.54,1171.6,4630.74,2018.05,10845.000000,0.09,2.862169e+05,175024.50
1,2014-11-01,338.649994,340.528992,321.054993,325.748993,16677200,33.118042,17.301759,-13.762845,37.900471,...,1.77,62.339950,80.54,1171.6,4630.74,2018.05,10845.000000,0.09,2.862169e+05,175024.50
2,2014-11-02,326.075012,329.049988,320.626007,325.891998,8603620,33.194650,16.667632,-14.688119,34.723531,...,1.89,49.099515,80.54,1171.6,4630.74,2018.05,10845.000000,0.09,2.934111e+05,174335.75
3,2014-11-03,325.569000,334.002014,325.480988,327.553986,12948500,34.138833,16.085732,-15.113083,41.224981,...,1.98,55.617203,78.78,1169.8,4638.91,2017.81,10808.669922,0.09,3.006052e+05,173647.00
4,2014-11-04,327.161011,331.766998,325.076996,330.492004,15655500,35.864492,15.414608,-15.039431,40.967179,...,1.95,25.086473,77.19,1167.7,4623.64,2012.10,10758.070313,0.09,3.077993e+05,171174.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,2024-03-27,69991.898438,71727.687500,68381.929688,69455.343750,40827113309,57.874623,3795.714961,2044.570728,49.459080,...,18.72,44.992788,81.35,2212.7,16399.52,5248.49,18255.240234,5.33,5.803127e+08,642478.50
3475,2024-03-28,69452.773438,71546.023438,68895.507813,70744.953125,34374900617,59.880894,3713.915008,2141.735613,57.053253,...,19.11,39.632276,83.17,2238.4,16379.46,5254.35,18312.669922,5.33,5.876795e+08,636953.00
3476,2024-03-29,70744.796875,70913.093750,69076.656250,69892.828125,25230851763,57.918033,3579.809472,2125.478921,62.244650,...,17.55,40.894306,83.17,2254.8,16379.46,5254.35,18312.669922,5.33,5.950462e+08,631427.50
3477,2024-03-30,69893.445313,70355.492188,69601.062500,69645.304688,17130241883,57.330162,3377.996630,2068.774755,65.389051,...,16.38,51.607467,83.17,2254.8,16379.46,5254.35,18312.669922,5.33,6.033108e+08,614851.00


In [5]:
#btc_gtrends_gnews_macro_block.to_csv('../Final_df/BTC_FINAL_DF.csv', index=False)

In [6]:
# Create one hot encoding for bullish sentiment and bearish sentiment
def categorize_sentiment(score):
    if score > 50:
        return "bullish"
    elif score >= 35:
        return "neutral"
    else:
        return "bearish"
    
btc_gtrends_gnews_macro_block['Sentiment_category'] = btc_gtrends_gnews_macro_block['Sentiment_Bullish'].apply(categorize_sentiment)

btc_gtrends_gnews_macro_block = pd.get_dummies(btc_gtrends_gnews_macro_block, columns=['Sentiment_category'], drop_first=False)

btc_gtrends_gnews_macro_block['Sentiment_category_bearish'] = btc_gtrends_gnews_macro_block['Sentiment_category_bearish'].astype(int)
btc_gtrends_gnews_macro_block['Sentiment_category_bullish'] = btc_gtrends_gnews_macro_block['Sentiment_category_bullish'].astype(int)
btc_gtrends_gnews_macro_block['Sentiment_category_neutral'] = btc_gtrends_gnews_macro_block['Sentiment_category_neutral'].astype(int)

In [8]:
btc_gtrends_gnews_macro_block.to_csv('../Final_df/BTC_FINAL_DF_ONE_HOT.csv', index=False)